<a href="https://colab.research.google.com/github/cosmic643/curiflow-learning-path/blob/prompt_engineering/llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!huggingface-cli login --token mytoken

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [1]:
!pip3 install promptify
!pip3 install deepeval
!pip3 install bitsandbytes

  Using cached tenacity-8.4.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached openai-1.51.2-py3-none-any.whl.metadata (24 kB)
  Using cached tiktoken-0.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached huggingface_hub-0.25.2-py3-none-any.whl.metadata (13 kB)
Using cached tenacity-8.4.2-py3-none-any.whl (28 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached openai-1.51.2-py3-none-any.whl (383 kB)
Using cached tiktoken-0.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached tqdm-4.66.5-py3-none-any.whl (78 kB)
Using cached huggingface_hub-0.25.2-py3-none-any.whl (436 kB)
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
      Successfully uninstalled tqdm-4.65.0
  Attempting uninstall: tenacity
    Found existing 

In [2]:
import torch

In [3]:
!pip3 install transformers

  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anthropic 0.3.4 requires pydantic<2.0.0,>=1.9.0, but you have pydantic 2.9.2 which is incompatible.
promptify 2.0.3 requires huggingface-hub==0.12, but you have huggingface-hub 0.25.2 which is incompatible.
promptify 2.0.3 requires openai==0.27.0, but you have openai 1.51.2 which is incompatible.
promptify 2.0.3 requires tenacity==8.2.2, but you have tenacity 8.4.2 which is incompatible.
promptify 2.0.3 requires tiktoken==0.4.0, but you have

In [4]:
# prompt: Load my model in 4 bit using bitsandbytes

from transformers import AutoModelForCausalLM, AutoTokenizer
import bitsandbytes as bnb
from transformers import BitsAndBytesConfig

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# Replace with your model ID
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", device_map="auto", torch_dtype=torch.float16, quantization_config=quant_config)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
!pip3 install langchain

In [6]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=512)

In [7]:
hf = HuggingFacePipeline(pipeline=pipe)

<ipython-input-7-8043894202af>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  hf = HuggingFacePipeline(pipeline=pipe)


In [8]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

In [9]:
prompt_template = PromptTemplate(
    input_variables=["input_text"],
    template="You are a helpful assistant. Answer the question: {input_text}"
)

# Step 3: Create an LLM chain with the prompt and the LLM
llm_chain = LLMChain(llm=hf, prompt=prompt_template)

# Step 4: Run the chain with user input
user_input = input("Enter your question: ")

# Pass the user input into the chain
response = llm_chain.invoke(input=user_input)

# Print the model's response
print(f"Assistant: {response}")

<ipython-input-9-a1f0e8109e38>:7: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=hf, prompt=prompt_template)


Enter your question: Explain me what are large language models


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant: {'input_text': 'Explain me what are large language models', 'text': 'You are a helpful assistant. Answer the question: Explain me what are large language models and their applications.\nLarge language models (LLMs) are a type of artificial intelligence (AI) that are designed to process and understand human language. They are trained on vast amounts of text data and can generate human-like responses to a wide range of questions and tasks. LLMs are typically based on deep learning techniques, such as transformers, which allow them to learn complex patterns in language.\n\nApplications of LLMs:\n\n1. **Chatbots and Virtual Assistants**: LLMs can be used to build chatbots and virtual assistants that can understand and respond to user queries, making them a key component of customer service and support systems.\n2. **Language Translation**: LLMs can be used to translate languages in real-time, making them a powerful tool for international communication and business.\n3. **Text Su

In [10]:
response = llm_chain.invoke(input=user_input)

# Print the model's response
print(f"Assistant: {response}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Assistant: {'input_text': 'Explain me what are large language models', 'text': 'You are a helpful assistant. Answer the question: Explain me what are large language models and their role in natural language processing.\nLarge language models (LLMs) are a type of artificial intelligence (AI) technology that is used to process and generate human-like language. They are a crucial component of natural language processing (NLP) and are used in a wide range of applications, including language translation, text summarization, and chatbots.\n\nHere\'s how LLMs work:\n\n1. **Training data**: LLMs are trained on massive datasets of text, which can include books, articles, websites, and social media posts. This training data is used to teach the model how to recognize patterns and relationships in language.\n2. **Neural networks**: LLMs are based on neural networks, which are composed of interconnected nodes or "neurons" that process and transmit information. In the case of LLMs, the neural netwo

In [11]:
!pip3 install "pydantic<2"

  Using cached pydantic-1.10.18-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (152 kB)
Using cached pydantic-1.10.18-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.9.2
    Uninstalling pydantic-2.9.2:
      Successfully uninstalled pydantic-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.15 requires pydantic>=2.7.0, but you have pydantic 1.10.18 which is incompatible.
langchain 0.3.3 requires pydantic<3.0.0,>=2.7.4, but you have pydantic 1.10.18 which is incompatible.
langchain-core 0.3.10 requires pydantic<3.0.0,>=2.5.2; python_full_version < "3.12.4", but you have pydantic 1.10.18 which is incompatible.
promptify 2.0.3 requires huggingface-hub==0.12, but you have huggingface-hub 0.25.2 which is incompatible.
promptif

In [12]:
from promptify import Prompter,OpenAI, Pipeline

sentence     =  """The patient is a 93-year-old female with a medical
                history of chronic right hip pain, osteoporosis,
                hypertension, depression, and chronic atrial
                fibrillation admitted for evaluation and management
                of severe nausea and vomiting and urinary tract
                infection"""

prompter     = Prompter('ner.jinja') # select a template or provide custom template
pipe         = Pipeline(prompter , hf)


result = pipe.fit(sentence, domain="medical", labels=None)

PydanticImportError: `pydantic.errors:ConfigError` has been removed in V2.

For further information visit https://errors.pydantic.dev/2.9/u/import-error

In [ ]:
!deepeval login